In [1]:
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import multiprocessing.popen_spawn_win32


## Notes
1. drop the missing value in bat_info.chemistry

## Questions


## Step 1: Read Data
1. Datasets: bat_info, batt_events,device_use_0,device_use_1
2. Library used: pandas, dask, (vaex)

Compare the performance for different libraries

In [2]:
# %%time
# bat_info = pd.read_csv('../data/batt_info.csv000',
#                        delimiter ="\1",
#                        error_bad_lines=False)
# bat_info.head()

In [3]:
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:11769 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 17.14 GB


In [4]:
%%time
bat_info = dd.read_csv('../data/batt_info.csv000',
                       delimiter ="\1",
                       assume_missing=True)
bat_info.head()

Wall time: 1.72 s


,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
0,2020-03-04 04:09:06,20200303-200228,2020030304-i-040171543e1bfef37-04zMFt2SzSU2O4Q...,V8BATTINFO_20200303130536.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-02 03:53:35,2020-03-03 04:05:53,2020-03-02 12:53:35,2020-03-03 13:05:53,2020-03-02 12:54:05,2020-03-02,1.0,LION,43092.0,43320.0,1.0
1,2020-03-04 04:09:06,20200303-200228,2020030304-i-040171543e1bfef37-04zMFt2SzSU2O4Q...,V8BATTINFO_20200303130536.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-02 03:53:35,2020-03-03 04:05:53,2020-03-02 12:53:35,2020-03-03 13:05:53,2020-03-02 12:57:12,2020-03-02,1.0,LION,43092.0,43320.0,1.0
2,2020-03-04 04:09:06,20200303-200228,2020030304-i-040171543e1bfef37-04zMFt2SzSU2O4Q...,V8BATTINFO_20200303130536.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-02 03:53:35,2020-03-03 04:05:53,2020-03-02 12:53:35,2020-03-03 13:05:53,2020-03-03 08:09:34,2020-03-03,-1.0,NaN,43092.0,43320.0,1.0
3,2020-03-05 04:07:50,20200304-200229,2020030406-i-0728cf8ab79db9e58-7s0vYpXS8HKQRVf...,V8BATTINFO_20200304151756.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-03 04:05:49,2020-03-04 06:18:15,2020-03-03 13:05:49,2020-03-04 15:18:15,2020-03-03 13:06:26,2020-03-03,1.0,LION,43092.0,43320.0,1.0
4,2020-03-05 04:07:50,20200304-200229,2020030406-i-0728cf8ab79db9e58-7s0vYpXS8HKQRVf...,V8BATTINFO_20200304151756.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-03 04:05:49,2020-03-04 06:18:15,2020-03-03 13:05:49,2020-03-04 15:18:15,2020-03-03 16:46:17,2020-03-03,-1.0,NaN,43092.0,43320.0,1.0


As showned above, dask is much faster than pandas in reading batt info. So we use dask to read other datasets 

In [5]:
%%time
bat_events = dd.read_csv('../data/batt_acdc_events.csv000',
                       delimiter ="\1",
                       assume_missing=True,
                        blocksize = "100000")
bat_events.head()

Wall time: 10.8 s


,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,system_power_state,event_type,duration,battery_percent_remaining,battery_minutes_remaining
0,2020-04-23 04:07:48,20200422-210232,2020042207-i-04a9a906e1cd5620c-kkPOjcyZXZqMOox...,V8BATTEVENTS_20200422093417.V8,000037b573c4463abc8cba503ac06045,2020-04-21 07:32:56,2020-04-22 07:34:41,2020-04-21 09:32:56,2020-04-22 09:34:41,2020-04-21 09:33:26,2020-04-21,POWERON,DC,810.408,75.0,31.616667
1,2020-04-23 04:07:48,20200422-210232,2020042207-i-04a9a906e1cd5620c-kkPOjcyZXZqMOox...,V8BATTEVENTS_20200422093417.V8,000037b573c4463abc8cba503ac06045,2020-04-21 07:32:56,2020-04-22 07:34:41,2020-04-21 09:32:56,2020-04-22 09:34:41,2020-04-21 09:46:56,2020-04-21,POWEROFF,DC,13.573,69.0,73.766670
2,2020-04-23 04:07:48,20200422-210232,2020042207-i-04a9a906e1cd5620c-kkPOjcyZXZqMOox...,V8BATTEVENTS_20200422093417.V8,000037b573c4463abc8cba503ac06045,2020-04-21 07:32:56,2020-04-22 07:34:41,2020-04-21 09:32:56,2020-04-22 09:34:41,2020-04-21 09:47:10,2020-04-21,POWERON,DC,173.669,69.0,73.766670
3,2020-04-23 04:07:48,20200422-210232,2020042207-i-04a9a906e1cd5620c-kkPOjcyZXZqMOox...,V8BATTEVENTS_20200422093417.V8,000037b573c4463abc8cba503ac06045,2020-04-21 07:32:56,2020-04-22 07:34:41,2020-04-21 09:32:56,2020-04-22 09:34:41,2020-04-21 09:50:03,2020-04-21,POWERON,DC,38471.500,67.0,503.700012
4,2020-04-23 04:07:48,20200422-210232,2020042207-i-04a9a906e1cd5620c-kkPOjcyZXZqMOox...,V8BATTEVENTS_20200422093417.V8,000037b573c4463abc8cba503ac06045,2020-04-21 07:32:56,2020-04-22 07:34:41,2020-04-21 09:32:56,2020-04-22 09:34:41,2020-04-21 20:31:15,2020-04-21,POWEROFF,DC,43907.823,8.0,0.000000


In [6]:
%%time
device_0 = dd.read_csv('../data/devuse_4known_device.csv000',
                       delimiter ="[\t\1]",
                       assume_missing=True)
device_0.head()

d:\python\lib\site-packages\dask\dataframe\io\csv.py:538: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  head = reader(BytesIO(b_sample), **kwargs)


Wall time: 2.17 s


,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,device,hw_name,name,duration,status
0,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-10 20:58:51,2019-11-10,GUID_DEVINTERFACE_DISPLAY_ADAPTER,PCI\VEN_8086&DEV_0F31&SUBSYS_14DD1043&REV_0E,Intel(R) HD Graphics,16571.0,4.0
1,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVINTERFACE_KEYBOARD,ACPI\VEN_ATK&DEV_3001,PC/AT Enhanced PS/2 Keyboard (101/102-Key),82117.0,4.0
2,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVINTERFACE_MOUSE,ACPI\VEN_ETD&DEV_0108,ASUS Touchpad,82117.0,4.0
3,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVICE_BATTERY,ACPI\VEN_PNP&DEV_0C0A,Microsoft ACPI-Compliant Control Method Battery,82117.0,4.0
4,2019-11-14 04:07:26,20191113-200224,2019111402-i-0728cf8ab79db9e58-fUgBvxtUfADzdPI...,V8DEVUSE4KNOWN_20191113183250.V8,0000e65f586a467ca242d6932782e01f,2019-11-11 00:21:39,2019-11-14 02:33:37,2019-11-10 16:21:39,2019-11-13 18:33:37,2019-11-11 21:00:33,2019-11-11,GUID_DEVINTERFACE_HID,HID\VEN_ATK&DEV_4001,HID-compliant wireless radio controls,82117.0,4.0


In [7]:
%%time
device_1 = dd.read_csv('../data/devuse_4known_device.csv001',
                       delimiter ="[\t\1]",
                       assume_missing=True)
device_1.head()

d:\python\lib\site-packages\dask\dataframe\io\csv.py:538: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  head = reader(BytesIO(b_sample), **kwargs)


Wall time: 2.14 s


,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,device,hw_name,name,duration,status
0,2020-05-12 03:57:47,20200511-200325,2020051113-i-04a9a906e1cd5620c-fSoHUWNirho4c7C...,V8DEVUSE4KNOWN_20200511155204.V8,af851b9c1b134eeb91c71a590789e80a,2020-05-10 13:52:12,2020-05-11 13:52:36,2020-05-10 15:52:12,2020-05-11 15:52:36,2020-05-10 22:48:53,2020-05-10,GUID_DEVINTERFACE_DISK,SCSI\DiskSAMSUNG_MZNLN512HMJP-000MAV2,SAMSUNG MZNLN512HMJP-000L7,24953.0,4.0
1,2020-05-12 03:57:47,20200511-200325,2020051113-i-04a9a906e1cd5620c-fSoHUWNirho4c7C...,V8DEVUSE4KNOWN_20200511155204.V8,af851b9c1b134eeb91c71a590789e80a,2020-05-10 13:52:12,2020-05-11 13:52:36,2020-05-10 15:52:12,2020-05-11 15:52:36,2020-05-10 22:48:53,2020-05-10,GUID_DEVINTERFACE_VOLUME,STORAGE\Volume,Volume,24953.0,4.0
2,2020-05-12 03:57:47,20200511-200325,2020051113-i-04a9a906e1cd5620c-fSoHUWNirho4c7C...,V8DEVUSE4KNOWN_20200511155204.V8,af851b9c1b134eeb91c71a590789e80a,2020-05-10 13:52:12,2020-05-11 13:52:36,2020-05-10 15:52:12,2020-05-11 15:52:36,2020-05-10 22:48:53,2020-05-10,GUID_DEVINTERFACE_IMAGE,SUSD_VISTA_2008_WIN7_2008R2_64,Samsung Universal Scan Driver,24953.0,4.0
3,2020-05-12 03:57:47,20200511-200325,2020051113-i-04a9a906e1cd5620c-fSoHUWNirho4c7C...,V8DEVUSE4KNOWN_20200511155204.V8,af851b9c1b134eeb91c71a590789e80a,2020-05-10 13:52:12,2020-05-11 13:52:36,2020-05-10 15:52:12,2020-05-11 15:52:36,2020-05-10 22:48:53,2020-05-10,GUID_DEVINTERFACE_SENSOR,Sensors\SensorsSimulatorDriver,Microsoft Visual Studio Location Simulator Sensor,24953.0,4.0
4,2020-05-12 03:57:47,20200511-200325,2020051113-i-04a9a906e1cd5620c-fSoHUWNirho4c7C...,V8DEVUSE4KNOWN_20200511155204.V8,af851b9c1b134eeb91c71a590789e80a,2020-05-10 13:52:12,2020-05-11 13:52:36,2020-05-10 15:52:12,2020-05-11 15:52:36,2020-05-10 22:48:53,2020-05-10,GUID_DEVINTERFACE_USB_DEVICE,USB\VID_04F2&PID_B52C&REV_0029,USB Composite Device,24953.0,4.0


## Step 2: bat_info

In [10]:
bat_info.head()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
0,2020-03-04 04:09:06,20200303-200228,2020030304-i-040171543e1bfef37-04zMFt2SzSU2O4Q...,V8BATTINFO_20200303130536.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-02 03:53:35,2020-03-03 04:05:53,2020-03-02 12:53:35,2020-03-03 13:05:53,2020-03-02 12:54:05,2020-03-02,1.0,LION,43092.0,43320.0,1.0
1,2020-03-04 04:09:06,20200303-200228,2020030304-i-040171543e1bfef37-04zMFt2SzSU2O4Q...,V8BATTINFO_20200303130536.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-02 03:53:35,2020-03-03 04:05:53,2020-03-02 12:53:35,2020-03-03 13:05:53,2020-03-02 12:57:12,2020-03-02,1.0,LION,43092.0,43320.0,1.0
2,2020-03-04 04:09:06,20200303-200228,2020030304-i-040171543e1bfef37-04zMFt2SzSU2O4Q...,V8BATTINFO_20200303130536.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-02 03:53:35,2020-03-03 04:05:53,2020-03-02 12:53:35,2020-03-03 13:05:53,2020-03-03 08:09:34,2020-03-03,-1.0,NaN,43092.0,43320.0,1.0
3,2020-03-05 04:07:50,20200304-200229,2020030406-i-0728cf8ab79db9e58-7s0vYpXS8HKQRVf...,V8BATTINFO_20200304151756.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-03 04:05:49,2020-03-04 06:18:15,2020-03-03 13:05:49,2020-03-04 15:18:15,2020-03-03 13:06:26,2020-03-03,1.0,LION,43092.0,43320.0,1.0
4,2020-03-05 04:07:50,20200304-200229,2020030406-i-0728cf8ab79db9e58-7s0vYpXS8HKQRVf...,V8BATTINFO_20200304151756.V8,0000a4f5d14942e9b7ec0846d084f229,2020-03-03 04:05:49,2020-03-04 06:18:15,2020-03-03 13:05:49,2020-03-04 15:18:15,2020-03-03 16:46:17,2020-03-03,-1.0,NaN,43092.0,43320.0,1.0


In [11]:
len(bat_info)

18642943

In [15]:
len(bat_info.designed_capacity==bat_info.full_charge_capacity)

18642943

#### load_ts
1. Min: 2019-09-02 04:08:22
2. Max: 2021-01-04 03:14:15

In [12]:
temp = dd.to_datetime(bat_info.load_ts,unit='ns')
temp.max().compute()

Timestamp('2021-01-04 03:14:15')

In [13]:
temp.min().compute()

Timestamp('2019-09-02 04:08:22')

In [14]:
del temp

#### Guid
1. unique id num: 56956
2. total length: 18642943

In [14]:
bat_info.guid.nunique().compute()

56959

In [15]:
len(bat_info)

18642943

#### Interval Start UTC

1. max: 2076??? 
2. min: 2020-01-05 00:00:03
3. clean the weird data

In [33]:
temp = dd.to_datetime(bat_info.interval_start_utc,unit='ns')
temp.max().compute()

Timestamp('2076-09-01 11:38:00')

In [34]:
temp.min().compute()

Timestamp('2020-01-05 00:00:03')

In [39]:
#get the weird datetime
bat_info[bat_info.interval_start_utc >'2022'].compute()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
55019,2019-09-02 04:08:22,20190901-210313,2019090111-i-040171543e1bfef37-ND9X2R3DO2DGkQC...,V8BATTINFO_20760901134350.V8,5056f119152b45bfac326140c8e409dd,2076-09-01 11:38:00,2076-09-01 11:44:03,2076-09-01 13:38:01,2076-09-01 13:44:03,2076-09-01 12:38:29,2076-09-01,1.0,LION,42880.0,33800.0,1.0


In [41]:
bat_info_clean = bat_info[bat_info.interval_start_utc <'2022']
len(bat_info_clean)

18642942

#### lnterval End
1. max is weird, need clean
2. min is also weird
3. only keep the data from 2019 to 2022

In [42]:
bat_info_clean.interval_end_utc.max().compute()

'9220-03-09 15:12:56'

In [43]:
bat_info_clean.interval_end_utc.min().compute()

'1900-02-01 00:00:00'

In [44]:
bat_info_clean[bat_info_clean.interval_end_utc >'2022'].compute()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
7999,2020-04-08 04:09:23,20200407-210227,2020040717-i-04a9a906e1cd5620c-jU6EjIDvBZHdasy...,V8BATTINFO_20850104000351.V8,001bd23e073041b583b23fc3047d3edf,2020-04-07 14:40:44,2085-01-03 18:34:12,2020-04-07 20:10:45,2085-01-04 00:04:12,2020-04-07 20:37:10,2020-04-07,-1.0,NaN,48000.0,27420.0,1.0
8000,2020-04-08 04:09:23,20200407-210227,2020040717-i-04a9a906e1cd5620c-jU6EjIDvBZHdasy...,V8BATTINFO_20850104000351.V8,001bd23e073041b583b23fc3047d3edf,2020-04-07 14:40:44,2085-01-03 18:34:12,2020-04-07 20:10:45,2085-01-04 00:04:12,2020-04-07 20:40:37,2020-04-07,1.0,NaN,48000.0,27420.0,1.0
8001,2020-04-08 04:09:23,20200407-210227,2020040717-i-04a9a906e1cd5620c-jU6EjIDvBZHdasy...,V8BATTINFO_20850104000351.V8,001bd23e073041b583b23fc3047d3edf,2020-04-07 14:40:44,2085-01-03 18:34:12,2020-04-07 20:10:45,2085-01-04 00:04:12,2020-04-07 20:11:15,2020-04-07,1.0,NaN,48000.0,27420.0,1.0
118449,2020-07-03 03:17:40,20200702-200228,2020070209-i-0728cf8ab79db9e58-6CkSUKoMM9CamqL...,V8BATTINFO_20841012025952.V8,000ec9915bdd4bb4a5f6649a16578c3b,2020-06-19 07:45:46,2084-10-12 01:00:05,2020-06-19 09:45:46,2084-10-12 03:00:05,2020-06-19 10:06:22,2020-06-19,-1.0,NaN,45000.0,36470.0,1.0
118450,2020-07-03 03:17:40,20200702-200228,2020070209-i-0728cf8ab79db9e58-6CkSUKoMM9CamqL...,V8BATTINFO_20841012025952.V8,000ec9915bdd4bb4a5f6649a16578c3b,2020-06-19 07:45:46,2084-10-12 01:00:05,2020-06-19 09:45:46,2084-10-12 03:00:05,2020-06-19 09:46:27,2020-06-19,1.0,NaN,45000.0,36470.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167406,2020-09-25 03:37:23,20200924-200228,2020092500-i-0728cf8ab79db9e58-DecWmlTDf1P6DFK...,V8BATTINFO_21161231230335.V8,223b76bf2e2e4abb97ead8ba4e400572,2020-08-31 21:06:53,2117-01-01 04:03:48,2020-08-31 17:06:53,2116-12-31 23:03:48,2020-08-31 19:58:28,2020-08-31,1.0,NaN,53153.0,51987.0,1.0
167407,2020-09-25 03:37:23,20200924-200228,2020092500-i-0728cf8ab79db9e58-DecWmlTDf1P6DFK...,V8BATTINFO_21161231230335.V8,223b76bf2e2e4abb97ead8ba4e400572,2020-08-31 21:06:53,2117-01-01 04:03:48,2020-08-31 17:06:53,2116-12-31 23:03:48,2020-08-31 17:11:01,2020-08-31,1.0,NaN,53153.0,51987.0,1.0
169179,2020-06-13 03:16:26,20200612-200216,2020061210-i-0728cf8ab79db9e58-4Mn92VmgbFtzYVy...,V8BATTINFO_20220612072648.V8,0a277c6fd6eb41cda7032914a0369c6b,2020-06-12 10:22:58,2022-06-12 10:27:02,2020-06-12 07:22:58,2022-06-12 07:27:02,2020-06-12 07:24:18,2020-06-12,-1.0,NaN,0.0,0.0,0.0
169180,2020-06-13 03:16:26,20200612-200216,2020061210-i-0728cf8ab79db9e58-4Mn92VmgbFtzYVy...,V8BATTINFO_20220612072648.V8,0a277c6fd6eb41cda7032914a0369c6b,2020-06-12 10:22:58,2022-06-12 10:27:02,2020-06-12 07:22:58,2022-06-12 07:27:02,2020-06-12 07:23:28,2020-06-12,1.0,NaN,48944.0,39991.0,1.0


In [45]:
bat_info_clean[bat_info_clean.interval_end_utc <'2019'].compute()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
43341,2020-12-14 03:08:45,20201213-190234,2020121401-i-040171543e1bfef37-XZRtJUkigk9gNbe...,V8BATTINFO_20201213203318.V8,3def371d02a6430da96e41fc1706b304,2020-12-12 01:53:37,1900-02-01 00:00:00,2020-12-11 20:53:37,1900-02-01 00:00:00,2020-12-11 21:05:19,2020-12-11,1.0,LION,18115.0,18115.0,1.0
43922,2020-12-22 03:08:15,20201221-190233,2020122114-i-04a9a906e1cd5620c-2WmvvcqAlxz4D6l...,V8BATTINFO_20201221165903.V8,ab0ae5374e334dadbaf672497c8cb24d,2020-12-20 14:54:43,1900-02-01 00:00:00,2020-12-20 16:54:44,1900-02-01 00:00:00,2020-12-20 18:55:40,2020-12-20,-1.0,NaN,47520.0,15671.0,1.0
43923,2020-12-22 03:08:15,20201221-190233,2020122114-i-04a9a906e1cd5620c-2WmvvcqAlxz4D6l...,V8BATTINFO_20201221165903.V8,ab0ae5374e334dadbaf672497c8cb24d,2020-12-20 14:54:43,1900-02-01 00:00:00,2020-12-20 16:54:44,1900-02-01 00:00:00,2020-12-21 14:36:45,2020-12-21,-1.0,NaN,47520.0,15671.0,1.0
43924,2020-12-22 03:08:15,20201221-190233,2020122114-i-04a9a906e1cd5620c-2WmvvcqAlxz4D6l...,V8BATTINFO_20201221165903.V8,ab0ae5374e334dadbaf672497c8cb24d,2020-12-20 14:54:43,1900-02-01 00:00:00,2020-12-20 16:54:44,1900-02-01 00:00:00,2020-12-20 16:55:31,2020-12-20,1.0,NaN,47520.0,15671.0,1.0
43925,2020-12-22 03:08:15,20201221-190233,2020122114-i-04a9a906e1cd5620c-2WmvvcqAlxz4D6l...,V8BATTINFO_20201221165903.V8,ab0ae5374e334dadbaf672497c8cb24d,2020-12-20 14:54:43,1900-02-01 00:00:00,2020-12-20 16:54:44,1900-02-01 00:00:00,2020-12-21 16:06:32,2020-12-21,1.0,NaN,47520.0,15671.0,1.0
43926,2020-12-22 03:08:15,20201221-190233,2020122114-i-04a9a906e1cd5620c-2WmvvcqAlxz4D6l...,V8BATTINFO_20201221165903.V8,ab0ae5374e334dadbaf672497c8cb24d,2020-12-20 14:54:43,1900-02-01 00:00:00,2020-12-20 16:54:44,1900-02-01 00:00:00,2020-12-21 07:38:33,2020-12-21,1.0,NaN,47520.0,15671.0,1.0
107749,2020-04-11 04:07:51,20200410-210232,2020041005-i-040171543e1bfef37-kGnCo6zMMxfCwFw...,V8BATTINFO_20200410110528.V8,03bc04e9e0c442828a8f7dfc41ec41d1,2020-04-09 05:33:48,1900-02-01 00:00:00,2020-04-09 11:03:48,1900-02-01 00:00:00,2020-04-09 17:45:13,2020-04-09,-1.0,NaN,41753.0,37087.0,1.0
107750,2020-04-11 04:07:51,20200410-210232,2020041005-i-040171543e1bfef37-kGnCo6zMMxfCwFw...,V8BATTINFO_20200410110528.V8,03bc04e9e0c442828a8f7dfc41ec41d1,2020-04-09 05:33:48,1900-02-01 00:00:00,2020-04-09 11:03:48,1900-02-01 00:00:00,2020-04-09 18:31:51,2020-04-09,-1.0,NaN,41753.0,37087.0,1.0
107751,2020-04-11 04:07:51,20200410-210232,2020041005-i-040171543e1bfef37-kGnCo6zMMxfCwFw...,V8BATTINFO_20200410110528.V8,03bc04e9e0c442828a8f7dfc41ec41d1,2020-04-09 05:33:48,1900-02-01 00:00:00,2020-04-09 11:03:48,1900-02-01 00:00:00,2020-04-09 11:04:31,2020-04-09,1.0,NaN,41753.0,37133.0,1.0
107752,2020-04-11 04:07:51,20200410-210232,2020041005-i-040171543e1bfef37-kGnCo6zMMxfCwFw...,V8BATTINFO_20200410110528.V8,03bc04e9e0c442828a8f7dfc41ec41d1,2020-04-09 05:33:48,1900-02-01 00:00:00,2020-04-09 11:03:48,1900-02-01 00:00:00,2020-04-10 10:56:24,2020-04-10,1.0,NaN,41753.0,37249.0,1.0


In [50]:
bat_info_clean = bat_info_clean[(bat_info.interval_start_utc <'2022')& (bat_info.interval_start_utc >'2019')]

#### datetime

In [52]:
bat_info_clean.dt.max().compute()

'9220-03-09'

In [53]:
bat_info_clean.dt.min().compute()


'2006-02-26'

In [54]:
bat_info_clean[bat_info_clean.dt >'2022'].compute()

,load_ts,batch_id,audit_zip,audit_internal_path,guid,interval_start_utc,interval_end_utc,interval_local_start,interval_local_end,ts,dt,battery_enum,chemistry,designed_capacity,full_charge_capacity,battery_count
98,2020-02-29 04:08:58,20200228-200229,2020022820-i-0728cf8ab79db9e58-RMXu43PROPRjAns...,V8BATTINFO_20760228212529.V8,00352bbfd0e84cf392b20690185bff6a,2020-02-28 20:06:37,2076-02-28 20:25:38,2020-02-28 21:06:37,2076-02-28 21:25:38,2049-02-28 21:07:05,2049-02-28,1.0,LION,53236.0,42940.0,1.0
58842,2020-02-08 04:09:28,20200207-200228,2020020705-i-04a9a906e1cd5620c-ruNtK4o7VSaIyg5...,V8BATTINFO_20320206013002.V8,0199d88562f54bdea72a4dd4a8673dd2,2020-02-05 17:32:16,2032-02-06 07:30:15,2020-02-05 11:32:16,2032-02-06 01:30:15,2032-02-06 01:27:44,2032-02-06,1.0,LIon,13409.0,13409.0,1.0
58849,2020-02-08 04:09:28,20200207-200228,2020020705-i-0728cf8ab79db9e58-EgUbYeJEHxHbbmI...,V8BATTINFO_20200206231143.V8,0199d88562f54bdea72a4dd4a8673dd2,2020-02-06 10:35:00,2032-02-06 07:31:56,2020-02-06 04:35:00,2032-02-06 01:31:56,2032-02-06 01:30:50,2032-02-06,1.0,LIon,13409.0,13409.0,1.0
86824,2020-02-24 04:08:52,20200223-200229,2020022318-i-0728cf8ab79db9e58-qWS3BdHP7fvIQXn...,V8BATTINFO_20200223132229.V8,1208fd8d4ff64a7a95ef455efe7629dc,2020-02-10 22:44:04,2020-02-23 18:22:38,2020-02-10 17:44:04,2020-02-23 13:22:38,2022-02-12 17:33:19,2022-02-12,1.0,LION,37296.0,31864.0,1.0
125108,2020-03-12 04:11:26,20200311-210227,2020031108-i-04a9a906e1cd5620c-AlLOGYCWOIUts10...,V8BATTINFO_21090121175923.V8,0220102874ca42c9a69f1c91aa158e16,2020-03-08 09:24:25,2109-01-21 14:59:31,2020-03-08 12:24:25,2109-01-21 17:59:31,2109-01-21 17:59:05,2109-01-21,1.0,LiON,46740.0,46740.0,1.0
172865,2020-03-10 04:14:15,20200309-210507,2020030919-i-040171543e1bfef37-dDhMo8Aqd6rZSr4...,V8BATTINFO_20200309141052.V8,01a40237387548f88e3580839385b383,2020-03-08 19:00:50,9220-03-09 15:12:56,2020-03-08 14:00:50,9220-03-09 10:12:56,9220-03-09 10:11:35,9220-03-09,-1.0,NaN,94618.0,94618.0,1.0
166556,2020-03-13 04:09:01,20200312-210228,2020031213-i-0728cf8ab79db9e58-jDmwt6jlnz2Cx0C...,V8BATTINFO_20411105000312.V8,8f3678806d864b54a9cbea6f16d291d6,2020-03-11 14:27:55,2041-11-04 17:06:26,2020-03-11 21:27:55,2041-11-05 00:06:26,2041-11-05 00:01:00,2041-11-05,-1.0,NaN,23760.0,163080.0,1.0
59767,2020-02-28 04:09:15,20200227-200230,2020022710-i-04a9a906e1cd5620c-KkRqX7pplhYKiuT...,V8BATTINFO_21200227205133.V8,95547b8c760b4b5b853f7bea3b20dffe,2020-02-27 04:34:32,2120-02-27 10:51:44,2020-02-27 14:34:32,2120-02-27 20:51:44,2120-02-27 20:44:11,2120-02-27,-1.0,NaN,50635.0,50635.0,1.0
105265,2020-02-20 04:10:20,20200219-200439,2020021907-i-0728cf8ab79db9e58-jzbw6XMGfE3Q0Nw...,V8BATTINFO_20500903000726.V8,e38e092f5e954f32924d51ef190b17e8,2020-02-14 02:45:54,2050-09-02 17:07:39,2020-02-14 09:45:55,2050-09-03 00:07:39,2050-09-03 00:05:12,2050-09-03,1.0,LIon,48000.0,34275.0,1.0
105267,2020-02-20 04:10:20,20200219-200439,2020021907-i-040171543e1bfef37-g37DcEGSRtkRFrr...,V8BATTINFO_20580203000351.V8,e38e092f5e954f32924d51ef190b17e8,2020-02-15 02:07:28,2058-02-02 17:04:16,2020-02-15 09:07:28,2058-02-03 00:04:16,2050-09-03 00:08:00,2050-09-03,1.0,LIon,48000.0,34275.0,1.0


In [55]:
bat_info_clean = bat_info_clean[(bat_info_clean.dt <'2022')& (bat_info_clean.dt >'2019')]

#### battery_enum

In [56]:
bat_info.battery_enum.value_counts().compute()

 1.0        11283563
-1.0         7124517
 2.0          234610
 3.0             252
 11324.0           1
Name: battery_enum, dtype: int64

In [57]:
bat_info_clean = bat_info_clean[bat_info_clean.battery_enum <10]

#### chemistry

In [58]:
bat_info.chemistry.value_counts().compute()

LION    1212253
LIon     529337
LiP      313785
Li-I     206591
PbAc      82922
Lion      77088
LiON      41649
MISS      37298
PBAC      33028
LiIo      20172
Real      19503
LiOn      15929
Li-i      14445
Li P      13323
           6988
LI-I       5313
Li         4817
NiMH       3364
PbAC       3303
LIP        2283
lith       2245
Lipo       1420
bq20       1097
4cel        717
PAD0        660
VBOX        367
NiMh        297
Unkn        232
Prim        224
LGI0        217
Fake        213
OYB0        204
LHP0         86
LIO          48
LiP7         19
BAT          12
Bad          12
Seal          9
PABA          4
����          3
id            3
\t            2
�             2
u             1
�          1
��--          1
             1
Virt          1
L��          1
Li�          1
LI��          1
LI�          1
L�          1
LiPP          1
Name: chemistry, dtype: int64

#### designed_capacity

In [49]:
bat_info_clean.designed_capacity.value_counts().compute()

0.0        760158
41440.0    690454
41998.0    537827
47520.0    419180
48840.0    392091
            ...  
50415.0         1
50406.0         1
50402.0         1
50401.0         1
42592.0         1
Name: designed_capacity, Length: 42071, dtype: int64

#### full_charge_capacity

In [59]:
bat_info_clean.full_charge_capacity.value_counts().compute()

0.000000e+00    757770
1.000000e+02    107307
4.294967e+09     49751
3.990000e+04     30150
4.674000e+04     23388
                 ...  
8.959200e+04         1
1.602200e+04         1
8.958600e+04         1
8.958200e+04         1
7.162100e+04         1
Name: full_charge_capacity, Length: 76767, dtype: int64

#### battery_count

In [60]:
bat_info_clean.battery_count.value_counts().compute()

1.0    17175039
0.0      866974
2.0      600147
3.0         758
Name: battery_count, dtype: int64

## Step 3: bat_events

In [4]:
bat_events.head()

NameError: name 'bat_events' is not defined

#### interval_start_utc

In [11]:
bat_events_clean = bat_events[(bat_events.interval_start_utc <'2022')& (bat_events.interval_start_utc >'2019')]

#### interval_end_utc

In [12]:
bat_events_clean = bat_events_clean[(bat_events_clean.interval_end_utc <'2022')& (bat_events_clean.interval_end_utc >'2019')]

#### dt	

In [13]:
bat_events_clean = bat_events_clean[(bat_events_clean.dt <'2022')& (bat_events_clean.dt >'2019')]

#### system_power_state

In [ ]:

bat_events_clean.system_power_state.value_counts().compute()

distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


#### duration

In [ ]:
bat_events_clean.duration.max().compute()

In [ ]:
bat_events_clean.duration.min().compute()

#### battery_percent_remaining

In [ ]:
bat_events_clean.battery_percent_remaining.max().compute()

#### battery_minutes_remaining

## Step 4: System info


In [6]:
%%time
sys_info = dd.read_csv('../data/system_sysinfo_unique_normalized.csv000.gz',
                       delimiter ="\1",
                       assume_missing=True)
sys_info.head()

d:\python\lib\site-packages\dask\dataframe\io\csv.py:477: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


Wall time: 981 ms


,load_ts,guid,chassistype,chassistype_2in1_category,countryname,countryname_normalized,modelvendor,modelvendor_normalized,model,model_normalized,...,cpu_suffix,screensize_category,persona,processor_line,vpro_enabled,firstreportdate,lastreportdate,discretegraphics,cpu_stepping,engagement_id
0,2020-10-01 00:48:15,10351643a4ae4b2d9cbc8a90db184050,Notebook,Unknown,Brazil,Brazil,Lenovo,Lenovo,Lenovo G480,Lenovo G480,...,Core-M,14x,Web User,M-Processor,N,2020-01-12 20:31:03,2020-09-29 18:30:56,N,Intel64 Family 6 Model 42 Stepping 7,iduu
1,2020-10-01 00:48:15,4f40b2f38cd74f6ea1559cebbc924619,Notebook,Unknown,Sweden,Sweden,Asus,Asus,T300CHI,T300CHI,...,Core-Y,12x,Web User,Unknown,N,2018-11-15 17:24:52,2020-09-28 12:49:29,N,Intel64 Family 6 Model 61 Stepping 4,iduu
2,2020-10-01 00:48:15,04b81eb4d661410ebff356f27fc544ed,Desktop,Unknown,Viet Nam,Viet Nam,System manufacturer,System manufacturer,System Product Name,System Product Name,...,Other,23x,Office/Productivity,Unknown,Y,2020-05-09 12:24:37,2020-09-29 08:54:35,Y,Intel64 Family 6 Model 158 Stepping 12,intel.com
3,2020-10-01 00:48:15,afd55a587fe14fc1904d572b3230dd54,Notebook,Unknown,Mexico,Mexico,HP,HP,HP ProBook 640 G2,HP ProBook 640 G2,...,Core-U,14x,Office/Productivity,U-Processor,N,2019-10-22 13:56:27,2020-09-28 08:41:47,N,Intel64 Family 6 Model 78 Stepping 3,intel.com
4,2020-10-01 00:48:15,4b3d6747c5f541949f4c1bd127b00bd9,Notebook,Unknown,Romania,Romania,Acer,Acer,Nitro AN515-52,Nitro AN515-52,...,Core-H,15x,Web User,H-Processor,N,2020-04-01 17:37:46,2020-04-23 19:34:16,Y,Intel64 Family 6 Model 158 Stepping 10,intel.com


In [11]:
sys_info.persona.value_counts().compute()

Web User                 35153
Casual User              12810
Gamer                    10611
Casual Gamer             10498
Office/Productivity       7519
Content Creator/IT        5883
Communication             5836
Win Store App User        4968
Entertainment             3768
File & Network Sharer     2231
Unknown                    723
Name: persona, dtype: int64

In [10]:
sys_info.columns

Index(['load_ts', 'guid', 'chassistype', 'chassistype_2in1_category',
       'countryname', 'countryname_normalized', 'modelvendor',
       'modelvendor_normalized', 'model', 'model_normalized', 'ram', 'os',
       '#ofcores', 'age_category', 'graphicsmanuf', 'gfxcard',
       'graphicscardclass', 'processornumber', 'cpuvendor', 'cpuname',
       'cpucode', 'cpu_family', 'cpu_suffix', 'screensize_category', 'persona',
       'processor_line', 'vpro_enabled', 'firstreportdate', 'lastreportdate',
       'discretegraphics', 'cpu_stepping', 'engagement_id'],
      dtype='object')

In [6]:
columns = ['chassistype_2in1_category', 'processornumber', 'cpu_suffix', 
            'processor_line','vpro_enabled','discretegraphics', 'cpu_stepping', 'engagement_id']

sys_info = sys_info[columns]
sys_info.head()

,chassistype_2in1_category,processornumber,cpu_suffix,processor_line,vpro_enabled,discretegraphics,cpu_stepping,engagement_id
0,Unknown,32 nm,Core-M,M-Processor,N,N,Intel64 Family 6 Model 42 Stepping 7,iduu
1,Unknown,14 nm,Core-Y,Unknown,N,N,Intel64 Family 6 Model 61 Stepping 4,iduu
2,Unknown,14 nm,Other,Unknown,Y,Y,Intel64 Family 6 Model 158 Stepping 12,intel.com
3,Unknown,14 nm,Core-U,U-Processor,N,N,Intel64 Family 6 Model 78 Stepping 3,intel.com
4,Unknown,14 nm,Core-H,H-Processor,N,Y,Intel64 Family 6 Model 158 Stepping 10,intel.com
